# Линейные коды

## Терминология

Линейный блоковый код это блоковый код, преобразующий $k$-мерное подпространство $\mathbb{F}_q^k$ в элементы $n$-мерного векторное пространства $\mathbb{F}_q^n$ над полем $\mathbb{F}_q$. Такой линейный код характеризуется длиной $n$, размерностью $k$ и арностью $q$. При $q = 2$ линейный блоковый код называется двоичным. Параметр $R = \frac{k}{n}$ называется скоростью кода.

Отображение $\operatorname{encoding} : \mathbb{F}^k_q \rightarrow \mathbb{F}^n_q$ описывает процессом кодирования, в то время как $\operatorname{decoding} : \mathbb{F}^n_q \rightarrow \mathbb{F}^k_q$ описывает процесс декодирования. При кодировании мы вносим избыточную информацию, добавляя проверочные $n - k$ символов. Эти же проверочные символы позволяют учитывать возможные ошибки в кодовых словах и исправлять их.

## Порождающая и проверочная матрица

Рассмотрим два вектора таких, что $w \in \mathbb{F}^k_q $ и $c \in \mathbb{F}^n_q$. Тогда преобразование можно рассматривать как умножение вектора на матрицу, а именно:

$$c = w \cdot \underset{k \times n}{G}$$

Матрица $G$ называется порождающей матрицей. Проверочной же матрицей называется такая матрица $H$, которая удовлетворяет условию:

$$\underset{k \times n}{G} \cdot \underset{n \times (n - k)}{H^T} = 0$$

Слово $c \in \mathbb{F}^n_q$ является кодовым тогда и только тогда, когда $c \cdot H^T = 0$.

На матрицы $G$ и $H$ накладывается ограничение, что они не содержат линейно зависимых строк или нулевых столбцов. В противном случае длина/размерность кода не может быть отождествлена с количеством столбцов/строк в матрице $G$. Всякую порождающую/проверочную матрицу кода можно привести к такому виду, чтобы выполнялись эти ограничения, и при этом они всё ещё оставались порождающими/проверочными матрицами этого же кода.

## Нахождение порождающей и проверочной матриц

Пусть у нас существуют матрицы $G'$ и $H'$ такие, что $G' = [I_k | S]$ и $H' = [-S^T | I_{n-k}]$, где $S$ — некоторая матрица, а $I_k$ — единичная матрица размерности $k$. Нетрудно показать, что $G' \cdot {H'}^T = 0$.

Тогда путём линейных преобразований можно научиться строить соответствующие матрицы для $G$ и $H$. Для построения $G'$ воспользуемся методом Гаусса. В процессе метода Гаусса мы будем выполнять линейные преобразования над строками (что не поменяет наш код) и перестановку столбцов. Тогда будем считать, что $G' = G \cdot P_G$, где $P_G$ — матрица перестановки. Аналогично и для $H'$ (в конце единичную матрицу можно переставить в конец). Для неё введём $P_H$ такую, что $H' = H \cdot P_H$.

Зная, что $(G \cdot P_G) \cdot (P_G^{-1} \cdot H^T) = G \cdot H^T = 0 = G' \cdot {H'}^T = (G \cdot P_G) \cdot {H'}^T$, получаем, что $(P_G^{-1} \cdot H^T) = {H'}^T$, откуда следует, что $H = H' \cdot P_G$.

Аналогично зная, что $(G' \cdot P_H) \cdot (P_H^{-1} \cdot {H'}^T) = G' \cdot {H'}^T = 0 = G \cdot H^T = G \cdot (P_H^{-1} \cdot {H'}^T)$, получаем, что $G = G' \cdot P_H$.

In [1]:
from algebraic.binary import binary_array

reed_muller_mt = binary_array([
    [1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 1, 1, 0, 0],
    [1, 0, 1, 0, 1, 0, 1, 0],
])
print(reed_muller_mt)

[[1 1 1 1 1 1 1 1]
 [1 1 1 1 0 0 0 0]
 [1 1 0 0 1 1 0 0]
 [1 0 1 0 1 0 1 0]]


In [2]:
from algebraic.linear import gen_to_check

h = gen_to_check(g=reed_muller_mt)
print(h)

[[1 1 1 1 0 0 0 0]
 [0 0 1 1 1 1 0 0]
 [1 0 1 0 1 0 1 0]
 [1 0 0 1 1 0 0 1]]


In [3]:
from algebraic.linear import check_to_gen

g = check_to_gen(h=reed_muller_mt)
print(g)

[[1 1 1 1 0 0 0 0]
 [0 0 1 1 1 1 0 0]
 [1 0 1 0 1 0 1 0]
 [1 0 0 1 1 0 0 1]]


## Минимальное расстояние

Расстояние Хэмминга между векторами $x$ и $y$ — количество позиций, в которых вектора $x$ и $y$ отличаются: $d_H(x, y) = | \{ i | x_i \neq y_i \} |$.

Пусть у нас есть код $\mathcal{C}$. Будем называть минимальным расстоянием $d = \min_{x \in \mathcal{C}, y \in \mathcal{C} \setminus \{x\}} d_H(x, y)$.

Минимальное расстояние кода равно $d$ тогда и только тогда, когда любое подмножество $C$ столбцов матрицы $H$ линейно независимо, если $|C| < d$, и при этом существует линейно зависимое подмножество, для которого $|C| \ge d$.

### Исправления и стирания

Минимальное расстояние кода говорит о том, насколько вектора из $encoding(\mathbb{F}^k_q)$ отличаются в пространстве $\mathbb{F}^n_q$. Если мы допустим $t$ ошибок в полученном кодовом слове, то мы можем отойти от исходного кодового слова на расстояние не большее, чем $t$, и при этом приблизится к другим кодовым словам на расстояние не больше, чем $t$. Если мы хотим, чтобы ошибки были исправлены, то необходимо, чтобы наш код имел расстояние $d \ge 2t + 1$ (но необязательно достаточно).

Также необходимо рассмотреть такую проблему как стирания. Если мы преобразуем наш вектор размера $n$ в вектор размера $n-v$, выкинув случайные элементы вектора, то тем самым отойдём от исходного кодового слова на расстояние $v$, но при этом ни к каким другим кодовым словам не приблизимся. Отсюда можно сделать вывод, что чтобы код с минимальным расстоянием $d$ исправляет $t$ ошибок и $v$ стираний необходимо, чтобы $d \ge 2t + v + 1$ (но необязательно достаточно).

## Ограничения линейных кодов

Опираясь на различные свойства кодов, можно определить некоторые взаимные ограничения на параметры $n$, $k$, $d$ и $q$. В дальнейшем большими буквами $N$, $K$ и т.д. будем обозначать функции соответствующих параметров, зависящие от прочих параметров.

Введём так же определение мощности кода $A_q$ как числа кодовых слов. Для линейного кода число слов равно $A_q(k) = q^k$.

### Граница Синглтона

Для любого блокового кода верно:

$$A_q(n, d) \le q^{n - d + 1}$$

In [4]:
from algebraic.linear.singleton import singleton_bound

max_value = singleton_bound(q=2, n=19, d=3)
print("A₂(19, 3) ≤ " + str(max_value))

A₂(19, 3) ≤ 131072


### Граница Хэмминга

Если для блокового кода верно, что $d = 2t + 1$, то:

$$A_q(n, d) \le \frac{q^n}{\sum_{i = 0}^{t} C_n^i (q - 1)^i}$$

In [5]:
from algebraic.linear.hamming import hamming_bound

max_value = hamming_bound(q=2, n=19, d=3)
print("A₂(19, 3) ≤ " + str(max_value))

A₂(19, 3) ≤ 26214.4


### Граница Варшамова-Гилберта

Существует блоковый код, удовлетворяющий неравенству:

$$A_q(n, d) \ge \frac{q^n}{\sum_{i=0}^{d-1} C_n^i (q-1)^i}$$

### Граница Варшамова-Гилберта для линейных кодов

Существует линейный блоковый код с числом слов $A_q(n, d) \ge q^k$, где $k$ — наибольшее целое число, такое что:

$$q^k \le \frac{q^n}{\sum_{i=0}^{d-2} C_{n-1}^i (q-1)^i}$$

In [6]:
from algebraic.linear.gilbert_varshamov import gilbert_varshamov_bound

max_value = gilbert_varshamov_bound(q=2, n=19, d=3)
print("A₂(19, 3) ≥ " + str(max_value))

A₂(19, 3) ≥ 16384


### Граница Грайсмера

Для двоичного линейного кода верно:

$$N(k, d) \ge d + N\Bigl(k - 1, \Bigl\lceil\frac{d}{2}\Bigr\rceil\Bigr)$$

In [7]:
from algebraic.linear.griesmer import griesmer_bound

max_value = griesmer_bound(k=14, d=5)
print("N(14, 5) ≥ " + str(max_value))

N(14, 5) ≥ 21


## Дуальный код

Дуальный код $\bar{\mathcal{C}}$ для линейного кода $\mathcal{C}$ это такой код, что $H_{\bar{\mathcal{C}}} = G_{\mathcal{C}}$. Отсюда следует, что $\forall c \in \mathcal{C}, \bar{c} \in \bar{\mathcal{C}} : c \cdot \bar{c} = 0$. Если $\mathcal{C} = \bar{\mathcal{C}}$, то такой код называется самодуальным.

### Преобразование Адамара для дуальных кодов

Допустим, есть некоторое отображение $f: \mathbb{F}_2^n \rightarrow \mathbb{R}$. Преобразованием Адамара этого отображения называется функция высшего порядка $\hat{f}(u) = \sum_{v \in \mathbb{F}_2^n}(-1)^{u v} \cdot f(v)$. Тогда выполняется следующее утверждение:

$$\sum_{\bar{c} \in \bar{\mathcal{C}}} f(\bar{c}) = \frac{1}{|\mathcal{C}|} \sum_{c \in \mathcal{C}} \hat{f}(c)$$.

Доказательство:

\begin{equation*}
\begin{gathered}
\frac{1}{|\mathcal{C}|} \sum_{c \in \mathcal{C}} \hat{f}(c) =
\frac{1}{|\mathcal{C}|} \sum_{c \in C} \sum_{w \in \mathbb{F}_2^n} (-1)^{<c, w>} f(w) =
\frac{1}{|\mathcal{C}|} \sum_{w \in \mathbb{F}_2^n} f(w) \sum_{c \in C}  (-1)^{<c, w>} = \\ =
\frac{1}{|\mathcal{C}|} \left(\sum_{\bar{c} \in \bar{\mathcal{C}}} f(\bar{c}) |C| + \sum_{w \in \mathbb{F}_2^n \setminus \bar{\mathcal{C}}} f(w) \sum_{c \in C}  (-1)^{<c, w>} \right) =
\frac{\sum_{\bar{c} \in \bar{\mathcal{C}}} f(\bar{c}) |\mathcal{C}|}{|\mathcal{C}|} =
\sum_{\bar{c} \in \bar{\mathcal{C}}} f(\bar{c})
\end{gathered}
\end{equation*}

Предпоследний переход верный, поскольку в любом двоичном коде $\mathcal{C}(n, k)$ количество слов $c \in \mathcal{C}$ таких, что $c_i = 1$ либо $0$, либо $2^{k - 1}$ (и аналогично для $c_i = 0$), а следовательно второе слагаемое в скобке будет равно $0$.
